# Load Environment variable

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

MY_ENV_VAR = os.getenv('MY_ENV_VAR')

In [2]:
# check if the environment variable is set
if MY_ENV_VAR is None:
    print('MY_ENV_VAR is not set')
else:
    print(f'MY_ENV_VAR is set!')

MY_ENV_VAR is set!


# Chat

In [3]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("MY_ENV_VAR"),)

def chat_gpt(prompt):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )

    return response.choices[0].message.content.strip()


## Load the train data

In [6]:
import json

# Load preprocessed data from JSON file
with open('/Users/home/Professor Zijun Yao Lab/GPT experiment/MIMIC III- Fine Tuning/preprocessed_data.json', 'r') as f:
    train_data = json.load(f)

# Use only the first 100 records
train_data = train_data[:100]

## Zero Shot - 1.0

In [8]:
# Create a prompt for each patient
def create_prompt(patient_features):
    age = patient_features['age']
    gender = patient_features['gender']
    diagnoses = ', '.join(patient_features['diagnoses'])
    
    prompt = f"""
    Patient Information:
    Age: {age}
    Gender: {gender}
    Diagnoses: {diagnoses}
    
    Based on the above information, what is the likelihood of mortality within the next year?
    """
    return prompt

prompts = [create_prompt(features) for features in train_data]

In [9]:
# generate prompts and make predictions
predictions = [chat_gpt(prompt) for prompt in prompts]

# Display the first 5 predictions
for i, prediction in enumerate(predictions[:5]):
    print(f'Prediction {i+1}: {prediction}')

# Save the predictions to a JSON file
with open('/Users/home/Professor Zijun Yao Lab/GPT experiment/Prompting/predictions.json', 'w') as f:
    json.dump(predictions, f)

In [15]:
# Generate prompts and make predictions
predictions = []

for record in train_data:
    prompt = create_prompt(record)
    prediction = chat_gpt(prompt)
    predictions.append({
        "prompt": prompt,
        "prediction": prediction,
        "actual": record["completion"]  # Assuming "completion" contains the true labels
    })

# Save the predictions
with open('predictions.json', 'w') as f:
    json.dump(predictions, f, indent=4)

print("Predictions saved to predictions.json")

Predictions saved to predictions.json


In [10]:
# print the first 5 predictions
for i, prediction in enumerate(predictions[:5]):
    print(f'Prediction {i+1}: {prediction["prediction"]}')
    print(f'Actual {i+1}: {prediction["actual"]}')
    print()

In [11]:
# if __name__ == '__main__':
#     while True:
#         prompt = input('You: ')
#         if prompt.lower() in ['exit', 'quit', 'bye']:
#             break

#         response = chat_gpt(prompt)
#         print(f'GPT-3: {response}')